# Chatbot para que me proponga que cenar


## Introducción

En este proyecto, se propone desarrollar un chatbot basado en el uso de una red neuronal para clasificar intents. El objetivo principal es crear un asistente conversacional capaz de interactuar con los usuarios y responder a sus solicitudes de manera adecuada. Para lograr esto, se utilizará un enfoque basado en machine learning, específicamente una red neuronal alimentada con un conjunto de datos etiquetados que describen diversos intents y patrones de conversación.

El chatbot se entrenará utilizando un archivo de configuración en formato JSON, que contiene un diccionario de intents. Cada intent incluye un conjunto de frases de ejemplo que representan las posibles entradas del usuario, así como una lista de respuestas predefinidas que el chatbot proporcionará al identificar dicho intent. Estos datos serán preprocesados y convertidos en una representación numérica que permita a la red neuronal aprender a clasificar correctamente los intents en función de las entradas del usuario.

El modelo entrenado será capaz de identificar el intent del usuario a partir de su mensaje y seleccionar una respuesta adecuada de las respuestas predefinidas asociadas.

El enfoque basado en el uso de redes neuronales para la clasificación de intents permite que el chatbot sea adaptable y escalable. A medida que se agreguen más intents o se modifiquen los existentes, el modelo se puede volver a entrenar para mejorar la precisión y la variedad de respuestas, proporcionando una experiencia más enriquecedora al usuario.



In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/dandev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/dandev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/dandev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:

nltk.download('stopwords', download_dir='nltk_data/')
nltk.download('punkt', download_dir='nltk_data/')


[nltk_data] Downloading package stopwords to nltk_data/...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to nltk_data/...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import json
import numpy as np
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
import unidecode

# Importar bibliotecas necesarias
import json
import numpy as np
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import unidecode



2024-10-15 21:03:04.171669: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-15 21:03:04.223864: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-15 21:03:04.225570: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-15 21:03:05.038362: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import json

# Cargar el archivo JSON
with open('intents.json', 'r', encoding='utf-8') as file:
    intents = json.load(file)

# Verificar el contenido del archivo
print(intents)


{'intents': [{'tag': 'saludo', 'patterns': ['Hola', 'Buenos días', '¿Qué tal?', '¿Cómo estás?', 'Hola, ¿qué tal?', '¡Hola!', 'Buenas', 'Hola, ¿cómo te va?', 'Saludos', 'Hola, ¿cómo te encuentras?', '¿Qué hay de nuevo?', 'Hola, mucho gusto', 'Buenas tardes', 'Hola, buen día', '¿Qué cuentas?'], 'responses': ['¡Hola! ¿En qué puedo ayudarte?', '¡Buenos días! ¿Cómo te puedo ayudar?', 'Hola, ¿en qué puedo asistirte?']}, {'tag': 'despedida', 'patterns': ['Adiós', 'Hasta luego', 'Nos vemos', 'Chao', 'Me tengo que ir', 'Hasta pronto', 'Cuídate', 'Nos vemos luego', 'Me voy', 'Que tengas un buen día', 'Hasta la próxima', 'Nos vemos más tarde', 'Adiós, que estés bien', 'Me despido', 'Que te vaya bien'], 'responses': ['¡Adiós! Que tengas un buen día.', 'Hasta luego, cuídate.', 'Nos vemos pronto.']}, {'tag': 'opcion', 'patterns': ['opción', 'Tienes otra opción', 'No me gusta tu sugerencia', 'otra cosa', 'tienes otra comida', '¿Qué otra opción hay?', '¿Puedes darme otra opción?', 'Dame otra sugerenci

La estructura del archivo JSON `intents` se organiza en forma de un diccionario que contiene una lista de "intents". Cada intent es un objeto que incluye tres elementos principales:

1. **`tag`**: Una etiqueta que identifica el nombre o categoría del intent. Sirve como identificador único para el intent.

2. **`patterns`**: Una lista de frases de ejemplo que representan posibles entradas del usuario. Estas frases son variaciones de lo que el usuario podría decir para expresar ese intent.

3. **`responses`**: Una lista de respuestas predefinidas que el chatbot puede utilizar cuando se detecta este intent. El chatbot seleccionará aleatoriamente una de estas respuestas para proporcionar una respuesta coherente al usuario.

### Ejemplo de la Estructura de `intents.json`

```json
{
  "intents": [
    {
      "tag": "saludo",
      "patterns": [
        "Hola",
        "Buenos días",
        "¿Qué tal?",
        "¿Cómo estás?",
        "Hola, ¿qué tal?"
      ],
      "responses": [
        "¡Hola! ¿En qué puedo ayudarte?",
        "¡Buenos días! ¿Cómo te puedo ayudar?",
        "Hola, ¿en qué puedo asistirte?"
      ]
    },
    {
      "tag": "despedida",
      "patterns": [
        "Adiós",
        "Hasta luego",
        "Nos vemos",
        "Chao",
        "Me tengo que ir"
      ],
      "responses": [
        "¡Adiós! Que tengas un buen día.",
        "Hasta luego, cuídate.",
        "Nos vemos pronto."
      ]
    }
  ]
}
```

En este ejemplo:
- Hay dos intents: "saludo" y "despedida".
- Cada intent tiene varias frases de ejemplo en `patterns` que el usuario podría decir.
- Las respuestas posibles se encuentran en `responses` y se elige una al azar cuando el intent se detecta.

Esta estructura permite que el chatbot reconozca diferentes intents basados en la entrada del usuario y responda adecuadamente.

In [5]:
import spacy
import unidecode
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.tokenize import word_tokenize

# Cargar el modelo de spaCy para español
nlp = spacy.load('es_core_news_sm')

# Cargar stop words en español
stop_words = set(stopwords.words('spanish'))

# Función para preprocesar el texto
def preprocess_text(text):
    # Quitar las tildes y convertir a minúsculas
    text = unidecode.unidecode(text.lower())

    # Corrección ortográfica usando TextBlob
    corrected_text = str(TextBlob(text).correct())

    # Tokenizar y lematizar usando spaCy
    doc = nlp(corrected_text)

    # Filtrar stop words, signos de puntuación y lematizar
    processed_words = [
        token.lemma_ for token in doc
        if token.text not in stop_words and not token.is_punct and not token.is_digit
    ]

    return ' '.join(processed_words)

# Paso 5: Preprocesar los patrones y las etiquetas
patterns = []
tags = []

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        processed_pattern = preprocess_text(pattern)
        patterns.append(processed_pattern)
        tags.append(intent["tag"])


ModuleNotFoundError: No module named 'spacy'

In [ ]:
# Paso 6: Convertir los patrones en características utilizando CountVectorizer
vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(patterns).toarray()

# Convertir las etiquetas en números utilizando LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(tags)

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

A continuación, vamos a crear una red neuronal densa para entrenarla utilizando los intents definidos en el archivo JSON. La red neuronal será capaz de aprender a clasificar las entradas del usuario en los diferentes intents, basándose en los ejemplos de frases proporcionados. Este modelo nos permitirá identificar el intent correcto y proporcionar una respuesta adecuada según las respuestas predefinidas asociadas a cada intent.

In [ ]:
# Paso 7: Crear el modelo de red neuronal
model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(set(tags)), activation="softmax"))

# Compilar el modelo
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

### Ejercicio
¿Creen que mejorarán las predicciones del chatbot si agregamos una capa recurrente?
___
Responder acá:




___

In [ ]:
# Entrenar el modelo
model.fit(X_train, y_train, epochs=200, batch_size=4, verbose=1, validation_data=(X_test, y_test))


In [ ]:
# Guardar el modelo y el vectorizador
model.save("chatbot_model.h5")
np.save("classes.npy", label_encoder.classes_)
np.save("vectorizer.npy", vectorizer)

 Los archivos se guardan de forma temporal en la raíz del entorno, es decir, en el directorio principal (`/content/`).

En este caso, los archivos se guardarán en:

- `/content/chatbot_model.h5`
- `/content/classes.npy`
- `/content/vectorizer.npy`

Tenga en cuenta que es de forma temporal, es decir si se reinicia el entorno estos se borran..

#### Cómo Descargar los Archivos Desde Colab

Puedes descargar los archivos a tu computadora con las siguientes líneas de código:

```python
from google.colab import files

# Descargar el modelo y los archivos relacionados
files.download("chatbot_model.h5")
files.download("classes.npy")
files.download("vectorizer.npy")
```

#### Guardar los Archivos en Google Drive

También puedes montar tu Google Drive y guardar los archivos allí para que permanezcan disponibles:

```python
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Guardar los archivos en una carpeta de Google Drive
model.save('/content/drive/MyDrive/chatbot_model.h5')
np.save('/content/drive/MyDrive/classes.npy', label_encoder.classes_)
np.save('/content/drive/MyDrive/vectorizer.npy', vectorizer)
```

De esta forma, los archivos se guardarán en tu Google Drive y estarán disponibles incluso después de cerrar la sesión en Colab.

In [ ]:
def chatbot_response(user_input):
    # Convertir la entrada del usuario en un formato que el modelo pueda usar
    input_data = vectorizer.transform([preprocess_text(user_input)]).toarray()
    prediction = model.predict(input_data)

    # Obtener la probabilidad más alta y su índice

    intent_index = np.argmax(prediction)
    max_prob = prediction[0][intent_index]  # Obtener la probabilidad más alta
    print(f"Probabilidad de la predicción: {max_prob:.2f}")
    # Definir un umbral de confianza
    confidence_threshold = 0.8  # 70% de confianza

    # Si la probabilidad es mayor que el umbral, devolver el intent; si no, pedir que repita la pregunta
    if max_prob >= confidence_threshold:
        intent_tag = label_encoder.inverse_transform([intent_index])[0]

        # Buscar una respuesta aleatoria para el intent predicho
        for intent in intents["intents"]:
            if intent["tag"] == intent_tag:
                response = random.choice(intent["responses"])
                break
    else:
        # Si la confianza es baja, pedir que repita la pregunta
        response = "No estoy seguro de haber entendido. ¿Podrías repetir la pregunta?"

    return response




In [ ]:

# Bucle de interacción del Chatbot
print("¡Hola! Soy un chatbot. ¿En qué puedo ayudarte? (Escribe 'salir' para terminar)")
while True:
    user_input = input("Tú: ")
    if user_input.lower() == "salir":
        print("Chatbot: ¡Hasta luego!")
        break
    response = chatbot_response(user_input)
    print("Chatbot:", response)

### Ejercicio:
Si prueban el chatbot y le preguntan "¿Qué otra opción hay?", notarán que no responde de forma correcta. Para solucionar esto, agreguen un nuevo intent en el archivo `intents.json` que maneje este tipo de solicitud, de modo que el chatbot pueda reconocerlo y responder adecuadamente. Agregar que si no entiende (o no encuentra la pregunta) responda "Repite la pregunta".

Como última modificación a nuestro chatbot, vamos a agregarle la opción de que nos pregunte si preferimos una opción vegetariana o con carne cuando estemos buscando recomendaciones de comida.

In [ ]:
preferencia = None  # Variable global para almacenar la preferencia del usuario
ultima_opcion = None  # Variable para recordar el último intent de cena

def chatbot_response1(user_input):
    global preferencia, ultima_opcion

    # Convertir la entrada del usuario en un formato que el modelo pueda usar
    input_data = vectorizer.transform([preprocess_text(user_input)]).toarray()
    prediction = model.predict(input_data)

    # Obtener la probabilidad más alta y su índice
    intent_index = np.argmax(prediction)
    max_prob = prediction[0][intent_index]  # Obtener la probabilidad más alta
    print(f"Probabilidad de la predicción: {max_prob:.2f}")

    # Definir un umbral de confianza
    confidence_threshold = 0.8  # 80% de confianza

    # Si la probabilidad es mayor que el umbral, devolver el intent
    if max_prob >= confidence_threshold:
        intent_tag = label_encoder.inverse_transform([intent_index])[0]

        # Si el intent es "cena" y aún no tenemos una preferencia, preguntar sobre la preferencia
        if intent_tag == "cena" and preferencia is None:
            preferencia = "preguntada"
            return "¿Prefieres una opción con carne o vegetariana?"

        # Si el intent es "otra_opcion", ofrecer otra opción basada en la preferencia ya seleccionada
        if intent_tag == "otra_opcion" and preferencia is not None:
            intent_tag = ultima_opcion  # Mantener la última preferencia de cena (carne o vegetariana)

        # Si el intent es de despedida o agradecimiento, restablecer la preferencia
        if intent_tag in ["despedida", "agradecimiento"]:
            preferencia = None  # Reiniciar la preferencia
            for intent in intents["intents"]:
                if intent["tag"] == intent_tag:
                    response = random.choice(intent["responses"])
                    return response

    # Si el chatbot ya preguntó sobre las preferencias
    if preferencia == "preguntada":
        if "carne" in user_input.lower():
            preferencia = "carne"
            intent_tag = "cena_carne"
        elif "vegetariana" in user_input.lower():
            preferencia = "vegetariana"
            intent_tag = "cena_vegetariana"
        else:
            return "Lo siento, no entendí tu preferencia. ¿Carne o vegetariana?"

    # Buscar una respuesta aleatoria para el intent predicho
    for intent in intents["intents"]:
        if intent["tag"] == intent_tag:
            response = random.choice(intent["responses"])
            break

    # Guardar el intent actual como la última opción seleccionada (carne o vegetariana)
    if intent_tag in ["cena_carne", "cena_vegetariana"]:
        ultima_opcion = intent_tag  # Mantener la preferencia para "otra opción"

    return response




In [ ]:
print("¡Hola! Soy un chatbot con memoria. ¿En qué puedo ayudarte? (Escribe 'salir' para terminar)")
while True:
    user_input = input("Tú: ")
    if user_input.lower() == "salir":
        print("Chatbot: ¡Hasta luego!")
        break
    response = chatbot_response1(user_input)
    print("Chatbot:", response)


In [ ]:
from google.colab import files

files.download("chatbot_model.h5")
files.download("classes.npy")
files.download("vectorizer.npy")

____
